<a href="https://colab.research.google.com/github/samp3209/capstone/blob/main/Blip_Mass_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and model download


In [1]:
#download model
import sys
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
    !git clone https://github.com/salesforce/BLIP
    %cd BLIP

Running in Colab.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.0 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.4-py3-none-any.whl size=292853 sha256=ddfad479a2dd91565165e4412dd2c37c20735b87fd92277757c477f775427913
  Stored in directory: /r

In [2]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#Link google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

# Define the path of the folder containing the images
folder_path = '/content/drive/MyDrive/MassOutput/Test/'

# Create an empty list to store the file paths of the images
file_paths = []

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
  # Check if the file is an image (ends with .jpg, .jpeg, or .png)
  if filename.endswith(('.jpg', '.jpeg', '.png')):
    # Get the file path of the image
    file_path = os.path.join(folder_path, filename)
    # Append the file path to the list
    file_paths.append(file_path)

# Print the file paths of the images
print(file_paths)

['/content/drive/MyDrive/MassOutput/Test/<PIL.Image.Image image mode=RGB size=512x512 at 0x7FDB29D6CB80>.png', '/content/drive/MyDrive/MassOutput/Test/<PIL.Image.Image image mode=RGB size=512x512 at 0x7FDB29D7BFD0>.png', '/content/drive/MyDrive/MassOutput/Test/<PIL.Image.Image image mode=RGB size=512x512 at 0x7FDB29D81F10>.png']


In [5]:
#opens image from path then stores to list as pil image
image_list = []
for path in file_paths:
  with Image.open(path).convert('RGB') as img:
      image_list.append(img)

In [6]:
#function for loading images and then transforming them to tensors for captioning
def load_images(raw_image):
  w,h = raw_image.size
  image_size = 512
  #display(raw_image.resize((w//5,h//5)))
    
  transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
  image = transform(raw_image).unsqueeze(0).to(device)   
  return image

In [7]:
#transforms inputs and stores them to list 
inputs = []
for image in image_list:
    x = load_images(image)
    inputs.append(x)

In [35]:
#Splits images into promotional and logos and dumps the rest in misc
chaunceys = []
logos = []
misc = []
from models.blip import blip_decoder
model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'
model = blip_vqa(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)
image_size = 512
for image in inputs:
        question='is subject a beaver?'
        question2='is subject a logo with a lake and or mountain?'
        with torch.no_grad():
          answer = model(image, question, train=False, inference='generate')
          answer2 = model(image, question2, train=False, inference='generate')
          if answer[0] == 'yes': 
            chaunceys.append('chauncey the beaver')
          elif answer2[0] == 'yes':
            logos.append('Champlain College Shield Logo')
          else:
            misc.append(answer)


reshape position embedding from 900 to 1024
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth


In [ ]:
# function for questioning promotional images
def promotional_questioning(promos):
  q1s = []
  q2s = []
  q3s = []
  q4s = []
  q5s = []
  from models.blip_vqa import blip_vqa

  image_size = 512
  model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'
  model = blip_vqa(pretrained=model_url, image_size=image_size, vit='base')
  model.eval()
  model = model.to(device)
  for image in promos:   
      question = 'what is the style of the image?'
      question2 = 'what are the colors predominant in the image?'
      question3 = 'how is the subject oriented?'
      question4 = 'what is the subject doing in this image?'
      question5 = 'where does this image take place?'
      with torch.no_grad():
          answer = model(image, question, train=False, inference='generate') 
          answer2 = model(image, question2, train=False, inference='generate') 
          answer3 = model(image, question3, train=False, inference='generate') 
          answer4 = model(image, question4, train=False, inference='generate') 
          answer5 = model(image, question5, train=False, inference='generate') 
          q1s.append(answer[0])
          q2s.append(answer2[0])
          q3s.append(answer3[0])
          q4s.append(answer4[0])
          q5s.append(answer5[0])

In [17]:
# function for questioning logos
def logo_questioning(logos):
  q1s = []
  q2s = []
  q3s = []
  q4s = []
  q5s = []
  from models.blip_vqa import blip_vqa

  image_size = 512
  model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'
  model = blip_vqa(pretrained=model_url, image_size=image_size, vit='base')
  model.eval()
  model = model.to(device)
  for image in logos:   
      question = 'what is the style of the image?'
      question2 = 'what are the colors predominant in the logo?'
      question3 = 'what famous artists style does this image image look like?'
      question4 = 'what is the shape of the logo?'
      question5 = 'what are the predominant features in the logo?'
      with torch.no_grad():
          answer = model(image, question, train=False, inference='generate') 
          answer2 = model(image, question2, train=False, inference='generate') 
          answer3 = model(image, question3, train=False, inference='generate') 
          answer4 = model(image, question4, train=False, inference='generate') 
          answer5 = model(image, question5, train=False, inference='generate') 

          print('answer: '+answer[0])
          q1s.append(answer[0])
          q2s.append(answer2[0])
          q3s.append(answer3[0])
          q4s.append(answer4[0])
          q5s.append(answer5[0])

reshape position embedding from 900 to 1024
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth
answer: cartoon
answer: cartoon
answer: cartoon


['blue', 'green', 'orange and blue']

['yes', 'yes', 'yes']

In [ ]:
new_captions = []
for i in range(len(captions)):
  nc =  captions[i] + ',' + q1s[i] + ',' + q2s[i] + ',' + q3s[i] + ',' + q4s[i] + ',' + q5s[i] + '.png'
  new_captions.append(nc)


In [ ]:
#matches the caption to the file path of the images to re name file to caption
path = '/content/drive/MyDrive/MassOutput/Test/'
for i in range(len(file_paths)):
  new_name = path + new_captions[i]
  old_name = file_paths[i]
  os.rename(old_name,new_name)